# 01 - Exploratory Data Analysis | SweetReturns Golden City

**Hacklytics 2026 -- SweetReturns: A candy-themed stock market analysis pipeline**

This is the first notebook in a 5-notebook pipeline. We perform exploratory data analysis on the **Massive Yahoo Finance Dataset** from Kaggle, covering ~500 tickers with 5 years of daily OHLCV data (Nov 2018 -- Nov 2023).

**Pipeline:**
1. `01_eda.ipynb` -- Exploratory Data Analysis (this notebook)
2. `02_feature_engineering.ipynb` -- Drawdowns, volatility, momentum indicators
3. `03_modeling.ipynb` -- Predictive modeling
4. `04_portfolio.ipynb` -- Portfolio optimization
5. `05_visualization.ipynb` -- Final dashboards and visualizations

**Dataset:** [Massive Yahoo Finance Dataset](https://www.kaggle.com/datasets/iveeaten3223times/massive-yahoo-finance-dataset)  
- File: `stock_details_5_years.csv` (65.58 MB)  
- ~500 tickers, ~630,000 rows  
- Columns: Date, Open, High, Low, Close, Volume, Dividends, Stock Splits, Company

In [ ]:
!pip install -q kagglehub pandas numpy matplotlib seaborn scipy

In [ ]:
import kagglehub
path = kagglehub.dataset_download("iveeaten3223times/massive-yahoo-finance-dataset")
print("Dataset downloaded to:", path)

In [ ]:
import pandas as pd
import numpy as np
import os

csv_path = os.path.join(path, "stock_details_5_years.csv")
df = pd.read_csv(csv_path, parse_dates=["Date"])
df.rename(columns={"Company": "ticker"}, inplace=True)
print(f"Shape: {df.shape}")
print(f"Tickers: {df['ticker'].nunique()}")
df.head()

In [ ]:
print("=== Data Types ===")
print(df.dtypes)
print(f"\n=== Missing Values ===")
print(df.isnull().sum())
print(f"\n=== Basic Statistics ===")
df.describe()

In [ ]:
ticker_coverage = df.groupby("ticker").agg(
    rows=("Date", "count"),
    start=("Date", "min"),
    end=("Date", "max"),
    avg_volume=("Volume", "mean")
).sort_values("rows", ascending=False)
print(f"Tickers with full coverage (>1200 days): {(ticker_coverage['rows'] > 1200).sum()}")
print(f"Tickers with partial coverage (<1000 days): {(ticker_coverage['rows'] < 1000).sum()}")
ticker_coverage.head(20)

In [ ]:
# GICS sector mapping for the top 500 stocks
SECTOR_MAP = {
    "Technology": ["AAPL", "MSFT", "NVDA", "GOOGL", "META", "AVGO", "ORCL", "CRM", "AMD", "ADBE",
                   "ACN", "CSCO", "INTC", "TXN", "QCOM", "INTU", "AMAT", "NOW", "IBM", "MU",
                   "LRCX", "ADI", "SNPS", "CDNS", "KLAC", "MRVL", "FTNT", "PANW", "CRWD", "MSI",
                   "NXPI", "APH", "TEL", "ROP", "KEYS", "ANSS", "CDW", "ZBRA", "EPAM", "PAYC",
                   "GOOG", "PLTR", "NET", "DDOG"],
    "Healthcare": ["UNH", "JNJ", "LLY", "ABBV", "MRK", "PFE", "TMO", "ABT", "DHR", "BMY",
                   "AMGN", "MDT", "ELV", "ISRG", "GILD", "CI", "SYK", "CVS", "REGN", "VRTX",
                   "BSX", "ZTS", "BDX", "HUM", "MCK", "HCA", "IDXX", "IQV", "EW", "DXCM",
                   "MTD", "A", "BIIB", "ALGN", "HOLX", "BAX", "TECH", "HSIC", "VTRS", "OGN"],
    "Financials": ["BRK.B", "JPM", "V", "MA", "BAC", "WFC", "GS", "MS", "BLK", "SPGI",
                   "AXP", "C", "SCHW", "CB", "MMC", "PGR", "ICE", "CME", "AON", "USB",
                   "MCO", "PNC", "AIG", "MET", "TFC", "AMP", "AFL", "TRV", "ALL", "PRU",
                   "BK", "COF", "FIS", "MSCI", "FI", "RJF", "WRB", "HBAN", "NTRS", "STT"],
    "Consumer Discretionary": ["AMZN", "TSLA", "HD", "MCD", "NKE", "LOW", "SBUX", "TJX", "BKNG", "CMG",
                                "MAR", "ORLY", "AZO", "ROST", "DHI", "YUM", "DG", "LEN", "EBAY", "GPC",
                                "ULTA", "APTV", "BBY", "GRMN", "POOL", "MGM", "DRI", "HAS", "WHR", "NVR",
                                "TSCO", "KMX", "WYNN", "CZR", "LVS", "RCL", "CCL", "NCLH", "HLT", "EXPE"],
    "Communication Services": ["GOOG", "META", "DIS", "CMCSA", "NFLX", "VZ", "T", "TMUS", "CHTR", "ATVI",
                                "EA", "WBD", "PARA", "FOX", "FOXA", "OMC", "IPG", "TTWO", "MTCH", "LYV",
                                "RBLX", "ZM", "PINS", "SNAP", "ROKU"],
    "Industrials": ["GE", "CAT", "HON", "UNP", "RTX", "BA", "UPS", "DE", "LMT", "ADP",
                    "MMM", "GD", "NOC", "ITW", "EMR", "WM", "ETN", "PH", "CTAS", "FAST",
                    "CSX", "NSC", "TT", "CARR", "OTIS", "RSG", "VRSK", "CPRT", "ODFL", "URI",
                    "DAL", "LUV", "UAL", "AAL", "FDX", "CHRW", "JBHT", "XYL", "AME", "ROK"],
    "Consumer Staples": ["PG", "KO", "PEP", "COST", "WMT", "PM", "MO", "MDLZ", "CL", "EL",
                          "KMB", "GIS", "SYY", "KHC", "HSY", "MKC", "K", "CPB", "SJM", "HRL",
                          "CAG", "TSN", "ADM", "BG", "TAP", "STZ", "BF.B", "CLX", "CHD", "WBA"],
    "Energy": ["XOM", "CVX", "COP", "SLB", "EOG", "MPC", "PXD", "PSX", "VLO", "OXY",
               "WMB", "HES", "DVN", "HAL", "BKR", "FANG", "TRGP", "KMI", "OKE", "CTRA",
               "MRO", "APA", "DEN", "EQT", "AR"],
    "Utilities": ["NEE", "DUK", "SO", "AEP", "D", "SRE", "EXC", "XEL", "ED", "WEC",
                   "ES", "AWK", "DTE", "PPL", "FE", "AEE", "CMS", "CNP", "ATO", "EVRG",
                   "NI", "PNW", "NRG", "LNT", "OGE"],
    "Real Estate": ["PLD", "AMT", "CCI", "EQIX", "PSA", "SPG", "O", "DLR", "WELL", "AVB",
                     "EQR", "VTR", "ARE", "MAA", "UDR", "ESS", "HST", "KIM", "REG", "PEAK",
                     "CPT", "SUI", "BXP", "INVH", "CUBE"],
    "Materials": ["LIN", "APD", "SHW", "ECL", "FCX", "NEM", "NUE", "VMC", "MLM", "DOW",
                   "DD", "PPG", "CE", "EMN", "IP", "PKG", "AVY", "ALB", "CF", "MOS",
                   "FMC", "IFF", "CTVA", "SEE", "RPM"],
}

# Invert to ticker -> sector
ticker_to_sector = {}
for sector, tickers in SECTOR_MAP.items():
    for t in tickers:
        ticker_to_sector[t] = sector

df["sector"] = df["ticker"].map(ticker_to_sector)
print(f"Mapped: {df['sector'].notna().sum()} / {len(df)} rows")
print(f"Unmapped tickers: {df[df['sector'].isna()]['ticker'].unique()[:20]}")
print(f"\n=== Stocks per Sector ===")
print(df.groupby("sector")["ticker"].nunique().sort_values(ascending=False))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle("SweetReturns \u2014 EDA Overview", fontsize=16, color="#FFD700")

# Close price distribution
axes[0,0].hist(df["Close"].dropna(), bins=100, color="#FF69B4", alpha=0.7, edgecolor="none")
axes[0,0].set_title("Close Price Distribution", color="#FFD700")
axes[0,0].set_xlabel("Price ($)")

# Volume distribution (log scale)
axes[0,1].hist(np.log10(df["Volume"].clip(lower=1)), bins=100, color="#00BFFF", alpha=0.7, edgecolor="none")
axes[0,1].set_title("Log10(Volume) Distribution", color="#FFD700")

# Rows per ticker
axes[1,0].hist(ticker_coverage["rows"], bins=50, color="#7FFF00", alpha=0.7, edgecolor="none")
axes[1,0].set_title("Trading Days per Ticker", color="#FFD700")
axes[1,0].set_xlabel("Number of rows")

# Average volume per sector
sector_vol = df.groupby("sector")["Volume"].mean().sort_values(ascending=True)
axes[1,1].barh(sector_vol.index, sector_vol.values, color="#9370DB", alpha=0.7)
axes[1,1].set_title("Mean Volume by Sector", color="#FFD700")

plt.tight_layout()
plt.savefig("eda_overview.png", dpi=150, bbox_inches="tight", facecolor="#1a1a2e")
plt.show()

In [ ]:
splits = df[df["Stock Splits"] != 0]
divs = df[df["Dividends"] != 0]
print(f"Stock split events: {len(splits)} across {splits['ticker'].nunique()} tickers")
print(f"Dividend events: {len(divs)} across {divs['ticker'].nunique()} tickers")
print(f"\nTop split events:")
splits.sort_values("Stock Splits", ascending=False).head(10)

In [ ]:
df = df.sort_values(["ticker", "Date"]).reset_index(drop=True)
df["daily_return"] = df.groupby("ticker")["Close"].pct_change()

return_stats = df.groupby("ticker")["daily_return"].agg(["mean", "std", "skew"]).dropna()
return_stats.columns = ["mean_return", "volatility", "skewness"]
return_stats["annualized_return"] = return_stats["mean_return"] * 252
return_stats["annualized_vol"] = return_stats["volatility"] * np.sqrt(252)
return_stats["sharpe"] = return_stats["annualized_return"] / return_stats["annualized_vol"]

print(f"=== Return Statistics (annualized) ===")
print(return_stats.describe())
print(f"\nTop 10 Sharpe Ratios:")
return_stats.sort_values("sharpe", ascending=False).head(10)

In [ ]:
# Save the cleaned DataFrame with sector labels and daily returns
df.to_parquet("stock_data_clean.parquet", index=False)
return_stats.to_parquet("return_stats.parquet")
print(f"Saved stock_data_clean.parquet: {df.shape}")
print(f"Saved return_stats.parquet: {return_stats.shape}")
print(f"\nReady for 02_feature_engineering.ipynb!")

## Summary
- **Dataset:** 500 tickers, ~630K rows, 5 years of daily OHLCV
- **Coverage:** Nov 2018 \u2014 Nov 2023
- **Sectors:** 11 GICS sectors mapped
- **Next:** Run `02_feature_engineering.ipynb` to compute drawdowns, volatility, momentum indicators